# ⚡️电交易市场分析与建模

> 本 Notebook 旨在系统介绍中国电力市场的运作背景、结构组成、价格机制、影响因素，并推荐若干模型用于电价预测与套利分析。
---

## 🧭 1. 市场背景介绍

近年来，中国电力市场正逐步向市场化方向推进，尤其是日前市场、现货市场的建设不断加快。在坚持“统一市场，两级运作”的方针下，不断深化上网电价改革，输配电价改革，销售电价改革。

**关键变化包括：**
- 推进电力现货市场试点（山西、广东、浙江等）
- 可再生能源接入带来不确定性提升
- 用户侧逐步开放，允许更多市场主体参与

**电力商品不同特性：**
- 现货交割必须时刻保持供需平衡
- 现货交割必须时刻满足电网安全约束
-   具备时间和空间的特性

📌 **市场参与者：**
- 发电企业（煤电、水电、风电、光伏等）
- 售电公司/电力中介
- 大型工业用户/聚合负荷
- 储能运营商
---

## 🧱 2. 市场结构概览
- 电力市场按照交易周期在时间上分为**中长期市场**和**现货市场**，其中现货市场具体细分为**日前交易**，**日内交易**和**实时交易**
- 电力市场在空间上形成“省内为基础，区域为过渡，全国统一为目标”的三级架构。

| 市场层级 | 交易时间               | 主要功能                 | 交易量占比 |
|----------|------------------------|--------------------------|------------|
| 日前市场 | 交割日前一天           | 锁定大部分电量，形成基准价格 | 70%-80%    |
| 日内市场 | 交割日当天每15-30分钟滚动 | 调整日前偏差             | 10%-15%    |
| 实时市场 | 交割前5分钟-1小时      | 平衡最后5%电量            | 5%-10%     |

---

## 💰 3. 电价组成机制
电力价格波动结果背后，是**边际成本定价原则**发挥的作用--系统根据报价从低到高调用机组，最后一台机组的报价即成为全市场结算基准。

电力价格= 能源边际成本 + 网络成本 + 损耗 + 拥堵价格 + 其他附加项

节点电价 = 各种成本综合叠加形成的“本地价格”

---


## 📈 4. 影响因素分析（供给侧视角）

> 为了准确预测日前市场每小时电价，尤其是基于供给侧数据，需全面识别影响电价的关键因素。立足于日前即可获取或合理预测的变量，结合电网调度机制、装机结构、可再生能源出力波动、经济指标及时间序列特征，构建高质量的模型输入特征空间。

### 🟢 1. 供给侧因子

| 因子名称 | 定义 | 数据来源 |
|----------|------|----------|
| **thermal_availability** <br> 火电可用率 (%) | 火电机组在日前计划中的可用容量占总装机容量的比例 | 电网调度计划（T-1） |
| **renew_volatility** <br> 新能源出力波动率（W/m²·min） | 风电和光伏在短时间内的功率变化率，衡量出力不稳定性 | Solcast API 或 CMA 格点预报 |
| **fuel_price** <br> 燃料价格（元/吨） | 标煤、天然气等主力燃料的市场价格 | 中国煤炭市场网、wind 数据终端 |
| **carbon_price** <br> 碳排放价格（元/吨） | ETS 市场交易价格，代表单位排放成本 | 上海环境能源交易所 |
| **congestion_risk** <br> 跨省输电阻塞风险 (%) | 跨区高压输电线路利用率或拥堵概率 | 调度中心日计划、实时调度平台 |

### 🔴 2. 需求侧因子

| 因子名称 | 定义 | 数据来源 |
|----------|------|----------|
| **load_forecast** <br> 系统负荷预测 (GW) | 预测全网日前各时段用电负荷 | 国家电网调度中心 |
| **industry_power** <br> 高耗能行业用电占比 (%) | 钢铁、化工等高耗能行业在总负荷中的实时用电占比 | 电网企业侧负荷监测平台 |
| **temperature** <br> 气温 (℃) | 反映主要区域日前时段的气温状况，影响空调采暖负荷。           | 气象局格点预报或气象台站观测 |
| **price_trend** <br> 电价 | 历史电价 | 本地历史价格 |

### 🟡 3. 市场与政策因子

| 因子名称 | 定义 | 数据来源 |
|----------|------|----------|
| **policy_cap** <br> 价格上限 (元/kWh) | 地区现货市场设定的最高成交限价 | 国家能源局、各省发改委发布文件 |
| **storage_action** <br> 储能行为 (GW) | 储能装置在 T-1 日的净充/放电功率 | 储能调控接口或电网储能调度系统 |
| **renewable_penalty_risk** <br> 弃风弃光风险预警 | 风电或光伏出力预测与调度限制的偏差指标 | 青海/甘肃新能源监控中心或自建规则计算 |
| **solar_irradiance** <br> 太阳辐射 (W/m²) | 太阳辐射强度，决定光伏发电基础出力水平。                    | 气象局观测数据或卫星遥感     |
| **wind_speed** <br> 风速 (m/s) | 反映风电场风力状况，辅助风电出力精准预测。                   | 气象局观测或风电场实测       | 

---


## 5.⏰ 时间特征编码方式（Time Feature Encoding）

为了捕捉电价的周期性和季节性波动，我们对时间相关特征进行了以下编码处理：

### 📌 1. 正余弦编码（周期型特征）
对于具有周期性的时间特征（如小时、星期几、月份），采用正余弦编码：

- **小时编码**：`hour_sin = sin(2π * hour / 24)`，`hour_cos = cos(2π * hour / 24)`
- **星期编码**：`weekday_sin = sin(2π * weekday / 7)`，`weekday_cos = cos(2π * weekday / 7)`
- **月份编码**：`month_sin = sin(2π * month / 12)`，`month_cos = cos(2π * month / 12)`

> 示例：1点（hour=1）与23点（hour=23）在时钟上相邻，但数值上差异大，直接数值编码会误导模型，正余弦编码可有效保留周期特性。

### 📌 2. 离散事件型特征（节假日/调休/高峰季节）
使用布尔值（0/1）编码如下信息：

- `is_weekend`: 是否为周末
- `is_holiday`: 是否为法定节假日

---

## 🤖 6. 推荐建模方法

### VMD+Bi-GRU+XGBoost
1.方法概述

电力价格序列通常包含多种时间尺度的波动成分，既有长期趋势，也有高频波动。为提升预测准确性，本文提出一种基于变分模态分解（Variational Mode Decomposition, VMD）的方法，将原始电价序列分解为：

- **长期趋势分量**：反映价格的平滑变化趋势，主要捕捉周期性和季节性信息；
- **高频波动分量**：反映短期尖峰、噪声及突发事件影响的价格波动。

随后，分别采用不同模型针对不同分量进行预测：

- 对长期趋势分量使用**GRU模型**进行建模，利用其对时间序列的长短期依赖捕捉能力；
- 对高频波动分量使用**XGBoost模型**进行建模，利用其非线性建模和特征选择优势；
- 最后将两部分预测结果叠加，得到完整的24小时电价预测序列。

!['VMD-bi-lstm-xgboost"](./deepseek_mermaid_20250617_0fd603.png)

### 



### Informer以及相关的变体
